In [1]:
import numpy as np
import pandas_datareader.data as web
import pandas as pd
import pandas_ta as ta
import pandas_market_calendars as mcal
from dotenv import load_dotenv
from datetime import datetime, timedelta
import matplotlib
import matplotlib.pyplot as plt
import yfinance as yf
import os

In [2]:
import requests
from bs4 import BeautifulSoup

def add_symbols_from_URL(URL='https://www.tradingview.com/markets/stocks-usa/market-movers-most-volatile/',companies = []):

    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")
    for row in soup.select('tbody tr'):
        if not row.find("td") == None:
            if not row.find("td").find("a") == None:
                #print(row.find("td").find("a").get_text())
                symbol = row.find("td").find("a").get_text()
                if symbol not in companies:
                    companies.append(row.find("td").find("a").get_text())

companies = []

f = open("S&P_Top50.txt", "r")
for lines in f:
    companies.append(lines.strip())

add_symbols_from_URL("https://www.tradingview.com/markets/stocks-usa/market-movers-most-volatile/",companies)
add_symbols_from_URL("https://www.tradingview.com/markets/stocks-usa/market-movers-gainers/",companies)
add_symbols_from_URL("https://www.tradingview.com/markets/stocks-usa/market-movers-losers/",companies)

In [3]:
NUM_OF_YEARS_PRED = 1 # num of year to predict
NUM_OF_YEARS_DATA = 5 # num of years of data to use

In [4]:
selected_features = []

file = open('selected_features.txt','r')
for lines in file:
	selected_features.append(lines.strip())

In [5]:
companies.sort()

In [6]:
def create_outcome(df:pd.DataFrame,year_pred:int,date:datetime):

    outcome_out = df.sort_index().copy()
    outcome_out.drop(columns=df.columns.difference(['date','symbol']),inplace=True)
    outcome_out.set_index(pd.DatetimeIndex(outcome_out["date"]),inplace=True)
    end_date =  df.index[-1]
    start_date = end_date.replace(year=end_date.year-year_pred)
    days_shift = (end_date - start_date).days

    outcome_colums = df.columns.difference(['date','symbol'])

    for i in range(1,days_shift+1):
        outcome = df[outcome_colums].shift(-i).add_suffix('_f'+str(i))
        outcome_out = pd.concat([outcome_out,outcome],axis=1)
         
    return outcome_out

def create_indicators(df:pd.DataFrame):
    features = df.sort_index().copy()

    features.set_index((features["date"]),inplace=True)

    features.ta.core = os.cpu_count()/2

    features.ta.strategy(ta.AllStrategy,exclude=["mcgd"],lookahead=False)
    features.ta.cdl_pattern(ta.ALL_PATTERNS, lookahead=False)

    features.drop(['open','high','low','close','volume'],axis=1,inplace=True)
    
    features.replace([np.inf, -np.inf], np.nan , inplace=True)
    for col in features.columns.difference(['date','symbol']): # fill nans
        features[col].interpolate(method="time",limit_direction="both",inplace=True)
        #features[col].interpolate(method="time",limit_direction="both",inplace=True)
    features.ffill(inplace = True)
         
    return features

def get_data(symbols,features,year_pred=NUM_OF_YEARS_PRED, year_data=NUM_OF_YEARS_DATA, data_source=None, begin_date=None,end_date=None):
    end_date = datetime.now().date() # to make sure we get todays data
    start_date = end_date.replace(year=end_date.year-(2*year_pred)-year_data)

    data_days = (end_date - start_date).days

    nyse = mcal.get_calendar('NYSE')
    dates = nyse.valid_days(start_date, end_date)

    while len(dates) <= data_days:
        if len(dates) == data_days:
            break
        start_date = start_date + timedelta(days=len(dates)-data_days)
        dates = nyse.valid_days(start_date, end_date)
    
    start_date = dates[0].date()
    end_date = dates[-1].date()

    days_shift = (end_date - end_date.replace(year=end_date.year-year_pred)).days

    features_out = pd.DataFrame()
    prices_out = pd.DataFrame()
    outcomes_out = pd.DataFrame()
    for symbol in symbols:
        if "." in symbol:
            continue

        ticker  = yf.Ticker(symbol)

        if ticker.info is None:
            continue

        df = ticker.history(start=start_date,interval='1d', auto_adjust=True)[['Open','High','Low','Close','Volume']].reset_index()     
        
        df.columns = ['date','open','high','low','close','volume']
        df['symbol'] = symbol

        if not df.date[0].date() == start_date:
            print("Not Enough Data For: ", symbol)
            continue
        
        #df['date'] = pd.to_datetime(df['date'])
        df = df.set_index(df['date'])
        #outcomes = create_outcome(df,year_pred,dates[-days_shift])
        #df = df[:dates[-days_shift]]
        features = create_indicators(df)
        

        df = df.set_index(['date','symbol'])
        features = features.set_index(['date','symbol'])
        #outcomes = outcomes.set_index(['date','symbol'])

        prices_out = pd.concat([prices_out,df],axis=0)
        features_out = pd.concat([features_out,features],axis=0)
        #outcomes_out = pd.concat([outcomes_out,outcomes],axis=0)

        print("Done Processing : ", symbol)

    return features_out.sort_index(),prices_out.sort_index()

In [7]:
#companies = ["MSFT","AAPL","AMZN"]
indicators_features,prices= get_data(companies,selected_features,year_pred=NUM_OF_YEARS_PRED,year_data=NUM_OF_YEARS_DATA)

130it [00:02, 49.65it/s]


Done Processing :  AA
Not Enough Data For:  AAMC


130it [00:02, 54.38it/s]


Done Processing :  AAME


130it [00:02, 53.17it/s]


...

Done Processing :  XOM
Not Enough Data For:  XOS
Not Enough Data For:  XTIA


130it [00:02, 44.93it/s]


Done Processing :  XXII
Not Enough Data For:  YMAB


130it [00:02, 50.05it/s]


Done Processing :  YTEN
Not Enough Data For:  ZNTL
Not Enough Data For:  ZPTA


In [10]:
features = pd.DataFrame(index=prices.index).sort_index()

features['close_pct_chnage'] = prices.close.pct_change()
features['volume_increase'] = np.where(prices.volume.shift(-1) > prices.volume, 1, 0)

In [11]:
features = pd.DataFrame(index=prices.index).sort_index()
features['f01'] = prices.close/prices.open-1 # daily return
features['f02'] = prices.open/prices.groupby(level='symbol', group_keys=False).close.shift(1)-1 
features['f03'] = prices.volume.apply(np.log) # log of daily volume
features['f04'] = prices.groupby(level='symbol', group_keys=False).volume.diff() # change since prior day
features['f05'] = prices.groupby(level='symbol', group_keys=False).volume.diff(50) # change since 50 days prior
pct_chg_fxn = lambda x: x.pct_change()
features['f06'] = prices.groupby(level='symbol', group_keys=False).volume.apply(pct_chg_fxn) # rate of change
# exponential moving avg of daily volume
# log of 5 day moving average of volume
ma_5 = lambda x: x.rolling(5).mean()
features['f07'] = prices.volume.groupby(level='symbol', group_keys=False).apply(ma_5).apply(np.log) 
# daily closing price vs. 10 day 
ema_10 = lambda x: x.ewm(span=10).mean()
features['f08'] = prices.close/ prices.close.groupby(level='symbol', group_keys=False).apply(ema_10)-1
# daily volume vs. 20 day moving average
ma_20 = lambda x: x.rolling(20).mean()
features['f09'] = prices.volume/ prices.volume.groupby(level='symbol', group_keys=False).apply(ma_20)-1
# daily volume vs. 40 day moving average
ma_40 = lambda x: x.rolling(40).mean()
features['f10'] = prices.volume/ prices.volume.groupby(level='symbol', group_keys=False).apply(ma_40)-1
# daily volume vs. 80 day moving average
ma_80 = lambda x: x.rolling(80).mean()
features['f11'] = prices.volume/ prices.volume.groupby(level='symbol', group_keys=False).apply(ma_80)-1
# daily volume vs. 160 day moving average
ma_160 = lambda x: x.rolling(160).mean()
features['f12'] = prices.volume/ prices.volume.groupby(level='symbol', group_keys=False).apply(ma_160)-1
# daily volume vs. 320 day moving average
ma_320 = lambda x: x.rolling(320).mean()
features['f12'] = prices.volume/ prices.volume.groupby(level='symbol', group_keys=False).apply(ma_320)-1

zscore_fun = lambda x: (x - x.rolling(window=200, min_periods=20).mean()) / x.rolling(window=200, min_periods=20).std()
features['f13'] =prices.groupby(level='symbol', group_keys=False).close.apply(zscore_fun)

rollrank_fxn = lambda x: x.rolling(200,min_periods=20).apply(lambda x: pd.Series(x).rank(pct=True)[0])
features['f14'] = prices.groupby(level='symbol', group_keys=False).volume.apply(rollrank_fxn)

features['f15'] = features['f11'].dropna().groupby(level='date').rank(pct=True) 
features['f16'] = features['f12'].dropna().groupby(level='date').rank(pct=True)

# money flow index (14 day)
features['f17'] = ta.volume.mfi(prices.high, prices.low, prices.close,prices.volume, window=14, fillna=False).astype(float)
# mean-centered money flow index
features['f18'] = features['f13'] - features['f13'].rolling(200,min_periods=20).mean()

#features.replace([np.inf, -np.inf], np.nan, inplace=True)
#n_bins = 10
#bin_fxn = lambda y: pd.qcut(y,q=n_bins,labels = range(1,n_bins+1))
#features['f19'] = prices.volume.groupby(level='symbol', group_keys=False).apply(bin_fxn)

features['f20'] = features['f06'].apply(np.sign)

plus_minus_fxn = lambda x: x.rolling(20).sum()
features['f21'] = features['f20'].groupby(level='symbol', group_keys=False).apply(plus_minus_fxn)

In [12]:
month_of_year = prices.index.get_level_values(level='date').month
one_hot_frame = pd.DataFrame(pd.get_dummies(month_of_year, dtype=int))
one_hot_frame.index = prices.index
# create column names 
feat_names = ['month_'+str(num) for num in list(range(1,1+12))]

# rename columns and merge
one_hot_frame.columns = feat_names

features = features.join(one_hot_frame)

features.replace([np.inf, -np.inf], np.nan, inplace=True)

features

f01       f02        f03        f04  \
date                      symbol                                             
2014-02-04 00:00:00-05:00 AA      0.016874       NaN  15.929367        NaN   
                          AAME    0.000000       NaN   8.594154        NaN   
                          AAPL    0.005812       NaN  19.746910        NaN   
                          ABBV    0.016320       NaN  16.077909        NaN   
                          ABT     0.009749       NaN  15.924195        NaN   
...                                    ...       ...        ...        ...   
2024-04-02 00:00:00-04:00 WFC    -0.000174 -0.004340  16.498715  3592400.0   
                          WMT    -0.012354 -0.001667  16.786279  5669300.0   
                          XOM     0.011533  0.007949  16.806813  6094600.0   
                          XXII   -0.044335  0.112451  12.196083   107180.0   
                          YTEN   -0.352941 -0.105263  14.776430  -317500.0   

                                         f05       f06        f07       f08  \
date                      symbol                                              
2014-02-04 00:00:00-05:00 AA             NaN       NaN        NaN  0.000000   
                          AAME           NaN       NaN        NaN  0.000000   
                          AAPL           NaN       NaN        NaN  0.000000   
                          ABBV           NaN       NaN        NaN  0.000000   
                          ABT            NaN       NaN        NaN  0.000000   
...                                      ...       ...        ...       ...   
2024-04-02 00:00:00-04:00 WFC    -10970500.0  0.325413  16.388109  0.000092   
                          WMT       331500.0  0.409703  16.575284 -0.018246   
                          XOM      -167700.0  0.441094  16.559940  0.036726   
                          XXII      132404.0  1.179981  11.874178 -0.040442   
                          YTEN     2504000.0 -0.108303  14.654238 -0.317242   

                                       f09       f10  ...  month_3  month_4  \
date                      symbol                      ...                     
2014-02-04 00:00:00-05:00 AA           NaN       NaN  ...        0        0   
                          AAME         NaN       NaN  ...        0        0   
                          AAPL         NaN       NaN  ...        0        0   
                          ABBV         NaN       NaN  ...        0        0   
                          ABT          NaN       NaN  ...        0        0   
...                                    ...       ...  ...      ...      ...   
2024-04-02 00:00:00-04:00 WFC    -0.214695 -0.228185  ...        0        1   
                          WMT     0.155908 -0.085626  ...        0        1   
                          XOM     0.134338  0.125970  ...        0        1   
                          XXII    1.193814  1.592689  ...        0        1   
                          YTEN   -0.620701 -0.631749  ...        0        1   

                                  month_5  month_6  month_7  month_8  month_9  \
date                      symbol                                                
2014-02-04 00:00:00-05:00 AA            0        0        0        0        0   
                          AAME          0        0        0        0        0   
                          AAPL          0        0        0        0        0   
                          ABBV          0        0        0        0        0   
                          ABT           0        0        0        0        0   
...                                   ...      ...      ...      ...      ...   
2024-04-02 00:00:00-04:00 WFC           0        0        0        0        0   
                          WMT           0        0        0        0        0   
                          XOM           0        0        0        0        0   
                          XXII          0        0        0        0        0

In [13]:
features = pd.concat([features,indicators_features],axis=1)
features.reset_index(level='symbol',inplace=True)
def interp_func(x):
    for col in x.columns.difference(['date','symbol','f19']):
        x[col].interpolate(method="time",limit_direction="both",inplace=True)
        x[col].ffill(inplace=True)
    return x

features = features.groupby('symbol', group_keys=False).apply(interp_func)
features.set_index([features.index,'symbol'],inplace=True)

In [14]:
features

f01       f02        f03         f04  \
date                      symbol                                              
2014-02-04 00:00:00-05:00 AA      0.016874 -0.000876  15.929367   6917354.0   
                          AAME    0.000000 -0.002532   8.594154       100.0   
                          AAPL    0.005812 -0.004383  19.746910 -48336400.0   
                          ABBV    0.016320 -0.003962  16.077909  -2619000.0   
                          ABT     0.009749 -0.003586  15.924195   -640100.0   
...                                    ...       ...        ...         ...   
2024-04-02 00:00:00-04:00 WFC    -0.000174 -0.004340  16.498715   3592400.0   
                          WMT    -0.012354 -0.001667  16.786279   5669300.0   
                          XOM     0.011533  0.007949  16.806813   6094600.0   
                          XXII   -0.044335  0.112451  12.196083    107180.0   
                          YTEN   -0.352941 -0.105263  14.776430   -317500.0   

                                          f05       f06        f07       f08  \
date                      symbol                                               
2014-02-04 00:00:00-05:00 AA        7052934.0  0.835418  16.083051  0.000000   
                          AAME          800.0  0.018519   8.416267  0.000000   
                          AAPL   -161915600.0 -0.128322  19.631806  0.000000   
                          ABBV     -2455100.0 -0.272639  15.801685  0.000000   
                          ABT       2715900.0 -0.077707  16.098960  0.000000   
...                                       ...       ...        ...       ...   
2024-04-02 00:00:00-04:00 WFC     -10970500.0  0.325413  16.388109  0.000092   
                          WMT        331500.0  0.409703  16.575284 -0.018246   
                          XOM       -167700.0  0.441094  16.559940  0.036726   
                          XXII       132404.0  1.179981  11.874178 -0.040442   
                          YTEN      2504000.0 -0.108303  14.654238 -0.317242   

                                       f09       f10  ...    VIDYA_14  \
date                      symbol                      ...               
2014-02-04 00:00:00-05:00 AA     -0.093025  0.061188  ...    0.391431   
                          AAME    1.699690  0.269009  ...    0.049315   
                          AAPL   -0.132850 -0.248186  ...    0.511884   
                          ABBV    0.004563 -0.282135  ...    1.924995   
                          ABT     0.081223 -0.334580  ...    3.558886   
...                                    ...       ...  ...         ...   
2024-04-02 00:00:00-04:00 WFC    -0.214695 -0.228185  ...   55.758652   
                          WMT     0.155908 -0.085626  ...   59.133032   
                          XOM     0.134338  0.125970  ...  111.881339   
                          XXII    1.193814  1.592689  ...    2.738130   
                          YTEN   -0.620701 -0.631749  ...    0.303951   

                                   VTXP_14   VTXM_14      VWAP_D     VWMA_10  \
date                      symbol                                               
2014-02-04 00:00:00-05:00 AA      1.069953  0.911178   25.838484   25.502111   
                          AAME    0.928571  1.000000    3.723672    3.716326   
                          AAPL    1.112036  0.816118   15.854253   16.585526   
                          ABBV    1.249115  0.772887   31.564819   32.687138   
                          ABT     1.370416  0.619804   29.853205   30.974900   
...                                    ...       ...         ...         ...   
2024-04-02 00:00:00-04:00 WFC     0.988263  0.989828   57.513332   57.339015   
                          WMT     0.834972  1.133047   59.380000   60.484668   
                          XOM     1.320778  0.509945  118.736666  115.058868   
                          XXII    0.892307  1.014902    1.955367    2.016870   
                          YTEN    0.824000  0.992000    0.26

In [15]:
prices

open        high         low  \
date                      symbol                                       
2014-02-04 00:00:00-05:00 AA       25.536102   26.103067   25.445389   
                          AAME      3.717398    3.745632    3.707987   
                          AAPL     15.818188   15.931075   15.721562   
                          ABBV     31.270976   31.794590   31.118532   
                          ABT      29.622171   30.067739   29.580912   
...                                      ...         ...         ...   
2024-04-02 00:00:00-04:00 WFC      57.360001   58.000000   57.189999   
                          WMT      59.900002   60.099998   58.880001   
                          XOM     117.919998  119.599998  117.330002   
                          XXII      2.030000    2.216100    1.710000   
                          YTEN      0.340000    0.350000    0.220000   

                                       close     volume  
date                      symbol                         
2014-02-04 00:00:00-05:00 AA       25.966997    8280108  
                          AAME      3.717398       5400  
                          AAPL     15.910123  376681200  
                          ABBV     31.781334    9606100  
                          ABT      29.910965    8237400  
...                                      ...        ...  
2024-04-02 00:00:00-04:00 WFC      57.349998   14631900  
                          WMT      59.160000   19506900  
                          XOM     119.279999   19911600  
                          XXII      1.940000     198012  
                          YTEN      0.220000    2614100  

[306840 rows x 5 columns]

In [16]:
features.sort_index().to_csv('RoboStockFeatures.csv')
prices.sort_index().to_csv('RoboStockPrices.csv')